# Data wrangling over excel files

---



In [2]:
!pip install polars 
!pip install xlsx2csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 4.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import polars as pl
import xlrd as xls
import numpy as np
import os

> ## **A prior implementation**: descriptions are included

In [ ]:
file = "/content/data_income/existencias-animales.xlsx"
file = "/content/data_income/actividad-principal-region.xlsx"
file = "/content/data_income/existencias-colmenas.xlsx"

In [ ]:
get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot]
get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
del_duplicate = lambda xs: [name for (name, pibot) in zip(xs, ["><"]+xs) if name != pibot]
## Filling up empty space within the row
filling_data = lambda xs: [ xs[i-1] if ( xs[i]=="" and i>0 ) else xs[i] for i in range(len(xs)) ]

Obtaining indexes for sectoring data content

In [ ]:
work_book = xls.open_workbook(file)
work_book = work_book.sheet_by_index(0)

column = 1

array_col = work_book.col_values(column)[1:]
#array_col = del_duplicate(array_col)

*a,ini,fin = get_indexes("", array_col)

## Filling up empty space within the row
array_col = filling_data(array_col)

## Keeping range that it cotains data
array_col = array_col[ini: fin]
print(array_col)

['Total Nacional - Región', 'Total Nacional', 'Región de Arica y Parinacota', 'Región de Tarapacá ', 'Región de Antofagasta', 'Región de Atacama', 'Región de Coquimbo', 'Región de Valparaíso', 'Región Metropolitana de Santiago', "Región del Libertador General Bernardo O'Higgins", 'Región del Maule', 'Región de Ñuble', 'Región del Biobío ', 'Región de La Araucanía', 'Región de Los Ríos', 'Región de Los Lagos', 'Región de Aysén del General Carlos Ibáñez del Campo', 'Región de Magallanes y de la Antártica Chilena']


Selecting columns as arrays
Sizing data of the columns
Data normalization
Storing arrays into fields

This process will finish when there are no more columns with data

In [ ]:
fields = np.array(array_col)

column=2
while True:
  try:
    array_col = work_book.col_values(column)[1:]
    #array_col = del_duplicate(array_col)
    
    ## Filling up empty space within the row
    array_col = filling_data(array_col)

    array_col = array_col[ini:fin]
    column+=1

    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.replace(array_col, " ", "_")

    #print(array_col)
    fields = np.row_stack((fields, array_col))
  except:
    print("Ending column")
    break
print(fields)

Ending column
[['Total Nacional - Región' 'Total Nacional'
  'Región de Arica y Parinacota' 'Región de Tarapacá '
  'Región de Antofagasta' 'Región de Atacama' 'Región de Coquimbo'
  'Región de Valparaíso' 'Región Metropolitana de Santiago'
  "Región del Libertador General Bernardo O'Higgins" 'Región del Maule'
  'Región de Ñuble' 'Región del Biobío ' 'Región de La Araucanía'
  'Región de Los Ríos' 'Región de Los Lagos'
  'Región de Aysén del General Carlos Ibáñez del Campo'
  'Región de Magallanes y de la Antártica Chilena']
 ['número_de_upa3,4' '5909.0' '6.0' '8.0' '7.0' '30.0' '297.0' '186.0'
  '209.0' '387.0' '686.0' '444.0' '947.0' '1690.0' '456.0' '536.0' '29.0'
  '1.0']
 ['número_de_colmenas' '333037.0' '53.0' '48.0' '25.0' '359.0' '6448.0'
  '24658.0' '29572.0' '50291.0' '63887.0' '38936.0' '36048.0' '38489.0'
  '16781.0' '27070.0' '366.0' '6.0']
 ['número_de_upa4' '4997.0' '5.0' '4.0' '5.0' '24.0' '230.0' '165.0'
  '188.0' '353.0' '597.0' '386.0' '824.0' '1402.0' '398.0' '402.

Removing rows with lots of null values to get title for the dataframe

In [ ]:
array_to_dataframe = np.transpose(fields)

pos = 0
normal_long = len(array_to_dataframe[pos])

while True:
  titles = array_to_dataframe[pos]
  out_arr = np.char.count(titles, sub ='', start=1, end=1)
  actual_long = np.sum(out_arr)
  if not len(out_arr) == actual_long :
    array_to_dataframe = array_to_dataframe[1:]
  else:
    break

titles = np.char.lower(titles)
titles = np.char.replace(titles, " ", "_")

for i in range(len(titles)):
  titles[i] = titles[i] +"_"+ str(i)

print(titles)

['total_nacional_-_región' 'número_de_upa3,4_1' 'número_de_colmenas_2'
 'número_de_upa4_3' 'número_de_colmenas_4' 'número_de_upa4_5'
 'número_de_colmenas_6']


Showing the datafram

In [ ]:
df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
pl.from_numpy
df_polar = df_polar[1:]
print(df_polar)

file = file.replace("_income","_outcome")
df_polar.write_csv(file.split('.')[0]+'.csv')


shape: (17, 7)
┌────────────┬────────────┬────────────┬──────────────┬──────────────┬──────────────┬──────────────┐
│ total_naci ┆ número_de_ ┆ número_de_ ┆ número_de_up ┆ número_de_co ┆ número_de_up ┆ número_de_co │
│ onal_-_reg ┆ upa3,4_1   ┆ colmenas_2 ┆ a4_3         ┆ lmenas_4     ┆ a4_5         ┆ lmenas_6     │
│ ión        ┆ ---        ┆ ---        ┆ ---          ┆ ---          ┆ ---          ┆ ---          │
│ ---        ┆ str        ┆ str        ┆ str          ┆ str          ┆ str          ┆ str          │
│ str        ┆            ┆            ┆              ┆              ┆              ┆              │
╞════════════╪════════════╪════════════╪══════════════╪══════════════╪══════════════╪══════════════╡
│ Total      ┆ 5909.0     ┆ 333037.0   ┆ 4997.0       ┆ 306758.0     ┆ 995.0        ┆ 26279.0      │
│ Nacional   ┆            ┆            ┆              ┆              ┆              ┆              │
├╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌

PanicException: ignored

> ## **Encapsulation and abstraction**: Object-oriented programming

In [5]:
class FileItem:
  def __init__(self, path):
    self.path = path
    self.list_files = []
  
  def collect_files(self):
    # Get the list of all files and directories
    self.list_files = os.listdir(self.path)
    self.list_files = [ self.path +'/'+ x for x in self.list_files ]

  def show_files(self):
    for x in range(len(self.list_files)):
      print(x+1, "  ", self.list_files[x])

In [7]:
class CsvFileItem:
  
  get_ini = lambda x, xs: [index+1 for (name, pibot, index) in zip(xs, ["><"]+xs, range(len(xs))) if name == pibot == '']
  get_indexes = lambda x, xs: [i for (y, i) in zip(xs, range(len(xs))) if x == y]
  del_duplicate = lambda xs: [name for (name, pibot) in zip(xs, ["><"]+xs) if name != pibot]
  ## Filling up empty space within the row
  filling_data = lambda xs: [ xs[i-1] if ( xs[i]=="" and i>0 ) else xs[i] for i in range(len(xs)) ]

  def __init__(self, path_file) -> None:
    self.path_file = path_file
    self.name_file = path_file.split('/')[-1]
    self.column=1
    self.dataframe=1
  
  def get_column_workbook(self, column):
    work_book = xls.open_workbook(self.path_file)
    work_book = work_book.sheet_by_index(0)
    array_col = work_book.col_values(column)[1:]

    return array_col

  def get_action_margin(self):
    array_col = self.get_column_workbook(1)
    *a,ini,fin = CsvFileItem.get_indexes("", array_col)
    return ini, fin

  def filling_empty_cells(self, array_col):
    ## Filling up empty space within the row
    array_col = CsvFileItem.filling_data(array_col)
    return array_col

  def data_normalization(self, array_col):
    ## Data normalization
    array_col = np.char.lower(array_col)
    array_col = np.char.strip(array_col)
    array_col = np.char.replace(array_col, " ", "_")
    array_col = np.char.replace(array_col, ".", "")
    array_col = np.char.replace(array_col, ",", "")
    return array_col
  

  def get_columns_with_data(self):
    ini, fin = self.get_action_margin()
    array_col = self.get_column_workbook(self.column)
    array_col = self.filling_empty_cells(array_col)
    array_col = self.filling_empty_cells(array_col)
    array_col = self.filling_empty_cells(array_col)
    array_col = array_col[ini:fin]


    column_collection = np.array(array_col)

    self.column+=1

    while True:
      try:
        array_col = self.get_column_workbook(self.column)
        array_col = self.filling_empty_cells(array_col)
        array_col = self.filling_empty_cells(array_col)
        array_col = self.filling_empty_cells(array_col)
        array_col = array_col[ini:fin]

        self.column+=1

        array_col = self.data_normalization(array_col)

        column_collection = np.row_stack((column_collection, array_col))
      except:
        # "Ending columns"
        break
    return column_collection
 
  def create_folder(self):
    path_outcome = self.path_file.split("/")[:-2] + ['data_outcome/']
    path_outcome = "/".join(path_outcome)
    
    # Check whether the specified path exists or not
    isExist = os.path.exists(path_outcome)
    if not isExist:
      # Create a new directory because it does not exist
      os.makedirs(path_outcome)
    
    return path_outcome

 
  def get_titles_dataframe(self):
    array_to_dataframe = np.transpose(self.get_columns_with_data())
    ## Going over rows
    titles=[]
    pos = 0
    normal_long = len(array_to_dataframe[pos])

    while True:
      titles = array_to_dataframe[pos]
      out_arr = np.char.count(titles, sub ='', start=1, end=1)
      actual_long = np.sum(out_arr)
      out_arr = out_arr.tolist()
      if not len(out_arr) == actual_long :
        array_to_dataframe = array_to_dataframe[1:]
      else:
        break

    #titles = self.data_normalization(titles)
    titles = titles.tolist()

    #if it would have many occurences, just in time 
    if len(titles) >0 :
      titles = [ name+"_"+str(i) for (name,i) in zip(titles, range(len(titles)))  ]

    return titles, array_to_dataframe


  def save_dataframe_csv(self):
    titles, array_to_dataframe = self.get_titles_dataframe()

    df_polar = pl.from_numpy(data=array_to_dataframe, columns=list(titles),)
    
    df_polar = df_polar[1:]
    self.dataframe = df_polar

    path_outcome = self.create_folder()
    path_outcome = path_outcome + self.name_file.split('.')[0]+'.csv'
    print(path_outcome)

    df_polar.write_csv(path_outcome)


## Class' instance for "FileItem" and "CsvFileItem"

In [7]:
files = FileItem('/content/data_income')
files.collect_files()
files.show_files()

1    /content/data_income/numero-upa-practicas-mejoramiento-suelo.xlsx
2    /content/data_income/numero-productores-tramos-edad.xlsx
3    /content/data_income/existencias-colmenas.xlsx
4    /content/data_income/numero-upa-orientación-colmenas.xlsx
5    /content/data_income/01_numero_superficie_de_upa_censadas_regional-xlsx.xlsx
6    /content/data_income/numero-superficie-de-upa-forestal-regional-comunal.xlsx
7    /content/data_income/numero-de-upa-y-superficie-por-categoría-de-uso-del-suelo.xlsx
8    /content/data_income/numero-productores-pueblos-originarios.xlsx
9    /content/data_income/numero-persona-natural-tipo-tenencia.xlsx
10    /content/data_income/tamaño-upa-región-comuna.xlsx
11    /content/data_income/actividad-principal-región.xlsx
12    /content/data_income/numero-superficie-de-upa-frutales-regional-comunal.xlsx
13    /content/data_income/numero-superficie-de-upa-aire-libre-bajo-cubierta-regional-comunal.xlsx
14    /content/data_income/numero-personas-administradoras.xlsx

In [ ]:
obj = CsvFileItem(files.list_files[0])
obj.save_dataframe_csv()
print(obj.dataframe)

In [ ]:
obj = CsvFileItem(files.list_files[1])
obj.save_dataframe_csv()
print(obj.dataframe)

In [ ]:
obj = CsvFileItem(files.list_files[2])
obj.save_dataframe_csv()
print(obj.dataframe.head())

In [ ]:
obj = CsvFileItem(files.list_files[3])
obj.save_dataframe_csv()
print(obj.dataframe.head())

In [ ]:
del obj
obj = CsvFileItem(files.list_files[4])
print(obj.path_file)

a, b=obj.get_action_margin()
print(a,b)

obj.get_columns_with_data()
print(obj.get_columns_with_data())

#print(list(map(list, zip(*obj.get_columns_with_data()))))


#obj.get_titles_dataframe()

#obj.save_dataframe_csv()
#print(obj.dataframe.head())
  

In [216]:
obj = CsvFileItem(files.list_files[5])
obj.save_dataframe_csv()
print(obj.dataframe.head())

/content/data_outcome/numero-superficie-de-upa-forestal-regional-comunal.csv
shape: (5, 5)
┌──────────────────┬────────────────┬────────────────┬───────────────────┬─────────────────────────┐
│ Total Nacional - ┆ comuna_4_5_1   ┆ especie_6_2    ┆ total             ┆ total                   │
│ Región_0         ┆ ---            ┆ ---            ┆ número_de_upa78_3 ┆ superficie_cultivo_(ha) │
│ ---              ┆ str            ┆ str            ┆ ---               ┆ _4                      │
│ str              ┆                ┆                ┆ str               ┆ ---                     │
│                  ┆                ┆                ┆                   ┆ str                     │
╞══════════════════╪════════════════╪════════════════╪═══════════════════╪═════════════════════════╡
│ Total Nacional   ┆ total_nacional ┆ total          ┆ 303640            ┆ 2068400835026186        │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤


In [ ]:



for index in range(5,len(files.list_files)):
  obj = CsvFileItem(files.list_files[index])
  obj.save_dataframe_csv()
  print(obj.dataframe.head())


## New procedures and methods for CsvFileItem class

In [ ]:
## Removing empty rows 
def remove_empty_rows(work_book):
  nrows = work_book.nrows
  ncols = work_book.ncols

  for i in range( nrows):
    nempty = work_book.count('')
    if nempty == ncols:
      work_book.remove_row(i)
  
  return work_book


## extextending values toward blank cells
def extending_values_blanckspaces(work_book):
  nrows = work_book.nrows
  ncols = work_book.ncols
  rack_rows = np.array(np.ones(ncols))
  for i in range(nrows):
    array_row = work_book.row_values(i)

    for index in range(len(array_row)):##going by columns
      if array_row[index] == '' and index > 0:
        work_book.row_values(i)[index] = work_book.row_values(i)[index-1]
  #     array_row[index] = array_row[index-1]
  #  array_row = np.array(array_row)
    
  #  rack_rows = np.row_stack( ( rack_rows , array_row ) )
  #rack_rows = rack_rows[1:]
  #return rack_rows
  return work_book 


## removing empty columns
def remove_empty_cols(work_book):
  nrows = work_book.nrows
  ncols = work_book.ncols
  
  for i in range(1, ncols):
    nempty = work_book[:, i-1:i]
    nempty = np.transpose(nempty).tolist()[0].count('')
    if float(nempty/nrows) >= 0.75 :
      work_book = work_book[:, i:]
  
  return work_book


## Verifying if this column is empty
def is_empty_col(work_book_column):
  ncols = len(work_book_column)
  nempty = work_book_column[:, 0:1]
  nempty = np.transpose(nempty).tolist()[0].count('')
  if nempty == ncols :
    return True
  else:
    return False
## Verifying if this row is empty
def is_empty_row(work_book_row):
  nempty = work_book_row.count('')
  if nempty == len(work_book_row):
    return True
  else:
    return False



## Removing comments head 1. By a row with contente between before and after blank line 2 and if it would be the first and unique column with data.
def remove_head_comments(work_book):
  nrows = work_book.nrows
  for index in range(0,len(work_book)):
    row = work_book[index]
    if not is_empty_row(row):
      if index == 0 and index < nrows-1:
        row_ahead = work_book[index+1]
        if is_empty_row(row_ahead):
          columns = len(work_book[index])
          work_book[index] = ['']*columns

      elif index > 0 and index < nrows-1:
        row_back = work_book[index-1]
        row_ahead = work_book[index+1]
        if is_empty_row(row_ahead) and is_empty_row(row_back)  :
          columns = len(work_book[index])
          work_book[index] = ['']*columns
  
  remove_empty_rows(work_book)
  return work_book


## Removing comments bottom 2. By couting from last row to up and verifying if it would be the first and unique column with data
##Pendient

def show_work_book(work_book):
  """ show work_rows' rows """
  for i in work_book.get_rows():
    print(i)

def to_ndimention_xls(work_book):
  """ Proccessing data and giving Numpy array format """
  rack_book = np.array(['']*work_book.ncols, dtype='<U32')
  for i in range(work_book.nrows):
    rack_book = np.vstack( ( rack_book, np.array(work_book.row_values(i), dtype='<U32') ) )
  return rack_book[1:]

################################################################################################################################################
print(files.list_files[7])
work_book = xls.open_workbook(files.list_files[7])
work_book = work_book.sheet_by_index(0)


for i in to_ndimention_xls(work_book):
  print(i.tolist().count(''))